In [ ]:
import pip
pip.main(['install', 'pandas'])
pip.main(['install', 'matplotlib'])
pip.main(['install', 'seaborn'])
pip.main(['install', 'scikit-learn'])

# CHARGEMENT DES BIBLIOTHEQUES

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

Chargement du jeu de données REF_MAGASIN

In [3]:
dataset_path = "C:\\Users\\belga\\OneDrive\\Bureau\\EMBA Big Data\\Module Datascience\\Projet Data Science S2 2024\\REF_MAGASIN.csv"
REF_MAGASIN = pd.read_csv(dataset_path, sep='|', low_memory=False)
print(REF_MAGASIN.shape)
print(list(REF_MAGASIN.columns))

(67, 4)
['CODESOCIETE', 'VILLE', 'LIBELLEDEPARTEMENT', 'LIBELLEREGIONCOMMERCIALE']


La ligne de code ci-dessous nous permet de voir le détail des colonnes de notre jeu de données ainsi que le type de données (chaîne de caractères, numérique)

In [4]:
REF_MAGASIN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   CODESOCIETE               67 non-null     object
 1   VILLE                     67 non-null     object
 2   LIBELLEDEPARTEMENT        67 non-null     int64 
 3   LIBELLEREGIONCOMMERCIALE  67 non-null     object
dtypes: int64(1), object(3)
memory usage: 2.2+ KB


Aperçu du jeu de données.

In [5]:
REF_MAGASIN

,CODESOCIETE,VILLE,LIBELLEDEPARTEMENT,LIBELLEREGIONCOMMERCIALE
0,MET,METZ-TESSY,74,Rhône-Alpes
1,ALB,GILLY SUR ISERE,73,Rhône-Alpes
2,ALM,LES MILLES,13,Littoral
3,AVI,VILLENEUVE-LES-AVIGNON,30,Littoral
4,BEA,BEAUMONT,63,Centre-Paris
...,...,...,...,...
62,STR,SISTERON,4,Littoral
63,NEV,NEVERS CEDEX,58,Centre-Paris
64,BSN,CESSON,77,Centre-Paris
65,CAG,CAGNES SUR MER,6,Littoral


Afin d'exploiter de façon optimale les informations, nous allons convetir le LIBELLE DEPARTEMENT en caractère. 

In [6]:
#transformation libellé département en object
REF_MAGASIN['LIBELLEDEPARTEMENT'] = REF_MAGASIN['LIBELLEDEPARTEMENT'].astype(str)

Vérification du traitement.

In [7]:
REF_MAGASIN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   CODESOCIETE               67 non-null     object
 1   VILLE                     67 non-null     object
 2   LIBELLEDEPARTEMENT        67 non-null     object
 3   LIBELLEREGIONCOMMERCIALE  67 non-null     object
dtypes: object(4)
memory usage: 2.2+ KB


Cette ligne de code nous permet d'identifier le nombre d'occurence de chaque CODESOCIETE, pour identifier les éventuels regroupements à effectuer. Nous allons également effectuer cette opération sur les autres colonnes de notre jeu de données.

In [8]:
#modalités variables:
REF_MAGASIN['CODESOCIETE'].value_counts(dropna = False)

CODESOCIETE
MET    1
VLG    1
SEY    1
SGL    1
SJV    1
      ..
PEP    1
PON    1
PRI    1
QUE    1
RMA    1
Name: count, Length: 67, dtype: int64

In [9]:
REF_MAGASIN['VILLE'].value_counts(dropna = False)

VILLE
METZ-TESSY             1
VILLE-LA-GRAND         1
SEYSSINS               1
SAINT-GENEST-LERPT     1
SAINT-JEAN-DE-VEDAS    1
                      ..
PERPIGNAN              1
LE PONTET              1
SAINT-PRIEST           1
QUETIGNY               1
RUEIL MALMAISON        1
Name: count, Length: 67, dtype: int64

In [10]:
REF_MAGASIN['LIBELLEREGIONCOMMERCIALE'].value_counts(dropna = False)

LIBELLEREGIONCOMMERCIALE
Rhône-Alpes       20
Littoral          19
Alsace-Est        15
Centre-Paris      12
Vente en ligne     1
Name: count, dtype: int64

In [11]:
REF_MAGASIN['LIBELLEREGIONCOMMERCIALE'].unique()

array(['Rhône-Alpes', 'Littoral', 'Centre-Paris', 'Alsace-Est',
       'Vente en ligne'], dtype=object)

Suppression de la modalité "vente en ligne", que nous n'allons pas exploiter.

In [12]:
# Créer un DataFrame de test avec la modalité 'Vente en ligne'
data = {'Region': ['Rhône-Alpes', 'Littoral', 'Centre-Paris', 'Alsace-Est', 'Vente en ligne']}
df = pd.DataFrame(data)

# Afficher le DataFrame avant la suppression
print("Avant la suppression :")
print(df)

# Supprimer la modalité 'Vente en ligne'
df = df[df['Region'] != 'Vente en ligne']

# Réinitialiser les index après la suppression
df.reset_index(drop=True, inplace=True)

# Afficher le DataFrame après la suppression
print("\nAprès la suppression :")
print(df)

Avant la suppression :
           Region
0     Rhône-Alpes
1        Littoral
2    Centre-Paris
3      Alsace-Est
4  Vente en ligne

Après la suppression :
         Region
0   Rhône-Alpes
1      Littoral
2  Centre-Paris
3    Alsace-Est


La ligne de code ci-dessous nous permet d'identifier les valeurs en doublons qui conviendra de traiter.

In [13]:
print(REF_MAGASIN.duplicated().sum())
REF_MAGASIN = REF_MAGASIN.drop_duplicates()

0


Nous procédons également à la vérification de présence ou non de valeurs manquantes (aucune valeur détectée).

In [14]:
pd.DataFrame(index=REF_MAGASIN.columns, columns=['%_missing_values'], data= (REF_MAGASIN.isna().sum().values / len(REF_MAGASIN)*100))

,%_missing_values
CODESOCIETE,0.0
VILLE,0.0
LIBELLEDEPARTEMENT,0.0
LIBELLEREGIONCOMMERCIALE,0.0


Le traitement des valeurs aberrantes ne concernant que les variables numériques, nous n'allons pas procéder à ce traitement car aucune de nos variables n'est concernées sur ce dataframe.

Nous renommons le "code société" en "magasin" pour faciliter la fusion de nos dataframes, par cette clé, par la suite.

In [15]:
REF_MAGASIN.rename(columns={'CODESOCIETE': 'MAGASIN'}, inplace=True)

Enregistrement de notre dataframe pour faciliter sa réutilisation dans un autre notebook, sans avoir à tout éxécuter. 

In [16]:
MAGASINS = "C:\\Users\\belga\\OneDrive\\Bureau\\EMBA Big Data\\Module Datascience\\Projet Data Science S2 2024\\MAGASINS_NET.pkl"
REF_MAGASIN.to_pickle(MAGASINS)